In [24]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_bar, plot_dynamic_trends, plot_line
from core.s3 import S3AssetManager
from core.render_svg import generate_sackoff_svg

import plotly.graph_objects as go

In [25]:
notebook_name = "fazenda_efecto_adiflow_en_sackoff"
s3 = S3AssetManager(notebook_name=notebook_name)

In [26]:
def compute_sackoff(
    df: pd.DataFrame,
    cols,

) -> pd.DataFrame:

    col_adiflow = "Tiene Adiflow"
    sackoff_name = "sackoff"
    group_cols = cols + [col_adiflow]

    df_group = (
        df
        .groupby(group_cols, dropna=False)
        .agg(
            diferencia=("diff", "sum"),
            reales=("peso_real", "sum"),
            production=("Producción (Ton)", 'sum'),
            anulation=("Anulación (Ton)", 'sum'),
            sackoff_mean=('sackoff_op', 'median'),
            ops=('op', 'nunique'),

            temp1_acond_c=('temp1_acond_c', 'mean'),

            pdi=('pdi', "mean"),
            pdi_agro=('pdi_agro', "mean"),

            finos=('finos', "mean"),
            finos_agro=('finos_agro', "mean"),

            dureza=('dureza', "mean"),
            dureza_agro=('dureza_agro', "mean")
        )
        .reset_index()
    )
    df_group[sackoff_name] = np.where(
        df_group["reales"] > 0,
        df_group["diferencia"] / df_group["reales"] * 100,
        np.nan
    )

    return df_group


def build_summary_table(summary_general_cut: pd.DataFrame) -> pd.DataFrame:
    df = summary_general_cut.copy()

    # Normalizar etiquetas por si vienen como 0/1, bool, etc.
    def norm_adiflow(x):
        s = str(x).lower()
        if "con" in s and "adiflow" in s:
            return "Con Adiflow"
        if "sin" in s and "adiflow" in s:
            return "Sin Adiflow"
        if s in ["1", "si", "sí", "true"]:
            return "Con Adiflow"
        if s in ["0", "no", "false"]:
            return "Sin Adiflow"
        return s

    df["Tiene Adiflow std"] = df["Tiene Adiflow"].apply(norm_adiflow)

    fila_con = df[df["Tiene Adiflow std"] == "Con Adiflow"].iloc[0]
    fila_sin = df[df["Tiene Adiflow std"] == "Sin Adiflow"].iloc[0]

    sackoff_sin = float(fila_sin["Sackoff Prom (%)"])
    sackoff_con = float(fila_con["Sackoff Prom (%)"])
    diff_sackoff = sackoff_con - sackoff_sin  # Con - Sin

    toneladas_con = float(fila_con["Producidas (Ton)"])
    toneladas_recuperadas = toneladas_con * diff_sackoff / 100.0

    resumen = {
        "Sackoff Sin Adiflow": round(sackoff_sin, 2),
        "Sackoff Con Adiflow": round(sackoff_con, 2),
        "Diferencia Sackoff": round(diff_sackoff, 2),
        "Toneladas Producidas Con Adiflow": round(toneladas_con, 2),
        "Toneladas Recuperadas": round(toneladas_recuperadas, 2),
    }

    return pd.DataFrame([resumen])



In [27]:

def build_monthly_summary_table(
    df: pd.DataFrame,
    month_col: str = "month",
    adiflow_col: str = "Tiene Adiflow",
    produced_col: str = "Producidas (Ton)",
    sackoff_col: str = "Sackoff (%)",
    month_order=None,
) -> pd.DataFrame:
    """
    Construye tabla resumen por mes:

    Mes | Sackoff Sin Adiflow | Sackoff Con Adiflow | Diferencia Sackoff |
        Toneladas Producidas Con Adiflow | Toneladas Recuperadas
    """

    data = df.copy()

    # Normalizar etiqueta Tiene Adiflow
    def norm_adiflow(x):
        s = str(x).lower()
        if "con" in s and "adiflow" in s:
            return "Con Adiflow"
        if "sin" in s and "adiflow" in s:
            return "Sin Adiflow"
        if s in ["1", "si", "sí", "true"]:
            return "Con Adiflow"
        if s in ["0", "no", "false"]:
            return "Sin Adiflow"
        return s

    data["Tiene Adiflow std"] = data[adiflow_col].apply(norm_adiflow)

    rows = []
    for m, g in data.groupby(month_col):
        fila_con = g[g["Tiene Adiflow std"] == "Con Adiflow"]
        fila_sin = g[g["Tiene Adiflow std"] == "Sin Adiflow"]

        if fila_con.empty or fila_sin.empty:
            # si falta alguno de los dos, saltamos el mes
            continue

        # Tomamos el valor (si hubiera más de una fila, usamos el primero;
        # puedes cambiar a sum/mean si hiciera falta)
        sackoff_con = float(fila_con[sackoff_col].iloc[0])
        sackoff_sin = float(fila_sin[sackoff_col].iloc[0])
        diff_sackoff = sackoff_con - sackoff_sin  # p.p.

        toneladas_con = float(fila_con[produced_col].iloc[0])
        toneladas_recuperadas = toneladas_con * diff_sackoff / 100.0

        rows.append({
            "Mes": m,
            "Sackoff Sin Adiflow": round(sackoff_sin, 2),
            "Sackoff Con Adiflow": round(sackoff_con, 2),
            "Diferencia Sackoff": round(diff_sackoff, 2),
            "Toneladas Producidas Con Adiflow": round(toneladas_con, 2),
            "Toneladas Recuperadas": round(toneladas_recuperadas, 2),
        })

    resumen = pd.DataFrame(rows)

    # Ordenar meses si se pasa un orden
    if month_order is not None and not resumen.empty:
        cat = pd.Categorical(resumen["Mes"], categories=month_order, ordered=True)
        resumen = resumen.assign(Mes=cat).sort_values("Mes").reset_index(drop=True)

    return resumen


In [28]:
#df_cap = s3.read_excel("raw/fazenda/SACK OFF FAZENDA.xlsx", sheet_name="CAP")
#df_sap = s3.read_excel("raw/fazenda/SACK OFF FAZENDA.xlsx", sheet_name="SAP")
df_cap = s3.read_excel("raw/fazenda/sackoff_fazenda_n8n.xlsx", sheet_name="CAP")
df_sap = s3.read_excel("raw/fazenda/sackoff_fazenda_n8n.xlsx", sheet_name="SAP")
df_sap = df_sap[df_sap["cerrada o abierta"] ==1]

In [29]:
df_sap.columns = [str(x).strip() for x in df_sap.columns]
columnas = [
'Orden', 'Material', 'Descripción',
'OP CAP', 'Cantidad planificada', 'Cantidad entregada',
'Unidad de medida',"101", "102",
"122", "309", "261",
"262", "641", "642",
'Difenrencia ent des', 'DIF PRO - CONS', 'SACKOFF %',
'CONS. CAP', 'SACOKFF CAP']
#df_sap.columns = df_cap.columns.map(str)
df_cap = df_cap[df_cap["O.P."].notnull()]
df_sap = df_sap[df_sap["Orden"].notnull()]

rename = {
    'Orden': 'order',
    'Descripción': "Dieta",
    'OP CAP': "op",
    'Fecha liberacion': "date",
    'Cantidad planificada': "panificadas",
    'Cantidad entregada': "entregadas",

     "101": "code_101",
    "102": "Anulación (Ton)",
    "122": "code_122",
    "309": "code_309",
    "261": "code_261",
    "262": "code_262",
    "641": "code_641",
    "642": "code_642",

    "PRODUCCION": "Producción (Ton)",
    'DIF PRO - CONS': "diff_prod",
    'SACKOFF %': "sackoff",
    'SACK OFF PROD': "sackoff_prod",
    'cerrada o abierta': "status"

}
df_sap = df_sap.rename(columns=rename)
df_sap_dep = df_sap[rename.values()]
cls_num = [
    "panificadas",
    "entregadas",
    "code_101",
    "Anulación (Ton)",
    "code_122",
    "code_309",
    "code_261",
    "code_262",
    "code_641",
    "code_642",
    "Producción (Ton)",
    "diff_prod",

]
df_sap_dep = df_sap_dep.copy()
for c in cls_num:
    df_sap_dep[c] = pd.to_numeric(df_sap_dep[c], errors="coerce")/1000
df_sap_dep["date"] = pd.to_datetime(df_sap_dep["date"])
df_sap_dep["month"] = df_sap_dep["date"].dt.to_period("M")

df_sap_dep["op"] = pd.to_numeric(df_sap_dep["op"], errors="coerce")

df_sap_dep["diff"] = df_sap_dep["code_101"] - df_sap_dep["Producción (Ton)"] - df_sap_dep["Anulación (Ton)"]
df_sap_dep["sackoff_op"] = df_sap_dep["diff"]/df_sap_dep["Producción (Ton)"]*100
df_sap_dep["sackoff_op"] = df_sap_dep["sackoff_op"].replace([np.inf, -np.inf], 0)

In [30]:
rename_cap = {"O.P.": "op", "Peso real": "peso_real", "Peso Agua": "peso_agua"}
df_cap = df_cap.rename(columns=rename_cap)
df_cap_dep = df_cap[rename_cap.values()].copy()
for c in ["peso_real", "peso_agua"]:
    df_cap_dep[c] = pd.to_numeric(df_cap_dep[c])/1000
df_cap_dep["op"] = pd.to_numeric(df_cap_dep["op"], errors="coerce")
df_cap_dep

,op,peso_real,peso_agua
0,20825.0,84.918095,0.77720
1,20826.0,34.398669,0.31485
2,20827.0,151.944689,1.38750
3,20828.0,151.872027,1.38780
4,20829.0,30.306526,0.27750
...,...,...,...
1126,22164.0,72.053364,0.00000
1127,22165.0,22.017223,0.00000
1128,22166.0,36.043390,0.00000
1129,22167.0,20.015533,0.00000


In [31]:
df = pd.merge(
    df_sap_dep,
    df_cap_dep,
    on='op',
    how='left',
)
df["Tiene Adiflow"] = np.where(df["peso_agua"]>0, "Con Adiflow", "Sin Adiflow")
df = df[df["op"].notnull()]
df["op"] = df["op"].astype(int).astype(str)

CUT_DATE_ENSAYO = '2025-09-01'
df = df[df["date"]>=CUT_DATE_ENSAYO]

In [32]:
#qa_agro = s3.read_excel("raw/fazenda/Base Fazenda.xlsx", sheet_name='CALIDAD AGROINDUSTRIA')
#qa_control = s3.read_excel("raw/fazenda/Base Fazenda.xlsx", sheet_name='3_control_prod_peletizado', skiprows=3)
qa_agro = s3.read_excel("raw/fazenda/bd_fazenda_n8n.xlsx", sheet_name='CALIDAD AGROINDUSTRIA')
qa_control = s3.read_excel("raw/fazenda/bd_fazenda_n8n.xlsx", sheet_name='3_control_prod_peletizado', skiprows=3)

In [33]:
qa_control

,Marca temporal,TURNO,SUPERVISOR,OPERARIO,PRODUCTO,OP CAP,TEMPERATURA DEL ACONDICIONADOR (°C) Pelet 1,TEMPERATURA DEL ACONDICIONADOR (°C) Pelet 2,DURABILIDAD (%),DUREZA (kg/cm2),...,% HUMEDAD,PAN,FECHA,HORA,RESPONSABLE DE CALIDAD,- PUNTO DE MUESTREO,- NÚMERO DE BACHE,- % HUMEDAD,- PAN,Fecha
0,2025-09-08 12:50:08.594,Turno 2,Jeffrey Marquez,Jose Urrego,NOVILLONAS SUPLEMENTO,21570.0,70.0,70.0,91.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-09-09 10:16:16.750,Turno 2,Jeffrey Marquez,Jose Urrego,GESTACION,21577.0,77.0,78.0,90.0,2.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-09-09 14:16:03.627,Turno 2,Jeffrey Marquez,Jose Urrego,GESTACION,21577.0,68.0,84.0,88.0,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-09-09 14:21:37.796,Turno 2,Jeffrey Marquez,Jose Urrego,GESTACION,21578.0,75.0,80.0,90.0,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-09-10 11:26:44.060,Turno 2,Jeffrey Marquez,Daniel Martinez,LEVANTE,21580.0,76.0,80.0,90.6,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2025-11-26 15:04:17.326,Turno 3,Luis Montenegro,Daniel Pastran,LEVANTE ESP P,22005.0,76.0,80.0,94.0,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,2025-11-26 16:39:12.423,Turno 3,Luis Montenegro,Edilberto Ortiz,LEVANTE ESP P,22005.0,76.0,80.0,92.5,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
237,2025-11-26 20:06:07.645,Turno 3,Luis Montenegro,Edilberto Ortiz,LEVANTE ESP P,22005.0,77.0,75.0,92.0,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,2025-11-27 08:29:59.144,Turno 2,Jeffrey Marquez,Daniel Pastran,PREINICIACION F2 P INMUNIDAD,22008.0,75.0,75.0,98.4,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
qa_control12 = qa_control.groupby(['OP CAP']).agg(
    product_name=('PRODUCTO', 'first'),
    temp1_acond_c=('TEMPERATURA DEL ACONDICIONADOR (°C) Pelet 1', 'mean'),
    #temp2_acond_c=('TEMPERATURA DEL ACONDICIONADOR (°C) Pelet 2', 'mean'),
    pdi=('DURABILIDAD (%)', 'mean'),
    dureza=('DUREZA (kg/cm2)', 'mean'),
    finos=('FINOS (%)', 'mean')
).reset_index().rename(columns={'OP CAP': 'op', 'PRODUCTO': 'product_name'})

qa_control3 = qa_control.groupby(['- OP CAP']).agg(
      product_name=('- PRODUCTO', 'first'),
    temp1_acond_c=('- TEMPERATURA DEL ACONDICIONADOR (°C) Pelet 3', 'mean'),
    pdi=('- DURABILIDAD (%)', 'mean'),
    dureza=('- DUREZA (kg/cm2)', 'mean'),
    finos=('- FINOS (%)', 'mean')
).reset_index().rename(columns={'- OP CAP': 'op',  '- PRODUCTO': 'product_name'})

qa_control_comp = pd.concat([qa_control12, qa_control3])


for cl in ['Durabilidad pellet ','Dureza pellet ', 'Finos pellet']:
    qa_agro[cl] = pd.to_numeric(qa_agro[cl], errors='coerce')

qa_agro_comp = qa_agro.groupby(['OP CAP']).agg(
    pdi_agro=('Durabilidad pellet ', 'mean'),
    dureza_agro=('Dureza pellet ', 'mean'),
    finos_agro=('Finos pellet', 'mean')).reset_index().rename(columns={'OP CAP': 'op', 'PRODUCTO': 'product_name'})


qa_control_comp["op"] = pd.to_numeric(qa_control_comp["op"], errors='coerce')
qa_control_comp = qa_control_comp[qa_control_comp["op"].notnull()]
qa_control_comp["op"] = qa_control_comp["op"].astype(int).astype(str)

qa_agro_comp["op"] =  pd.to_numeric(qa_agro_comp["op"], errors='coerce')
qa_agro_comp = qa_agro_comp[qa_agro_comp["op"].notnull()]
qa_agro_comp["op"] = qa_agro_comp["op"].astype(int).astype(str)

# TODO, bad
qa_control_comp[qa_control_comp.duplicated(subset=['op'], keep=False)]


,op,product_name,temp1_acond_c,pdi,dureza,finos
1,21577,GESTACION,72.500000,89.000000,3.000000,11.000000
7,21598,FINALIZACION,75.000000,91.000000,3.100000,9.000000
8,21599,FINALIZACION,76.000000,93.000000,3.000000,7.000000
18,21634,GESTACION,77.000000,90.000000,3.000000,10.000000
25,21669,GESTACION,75.000000,91.600000,3.250000,8.400000
28,21682,LEVANTE,80.000000,93.600000,3.200000,6.700000
29,21686,FINALIZACION,75.000000,92.000000,3.000000,7.000000
32,21704,LEVANTE,78.000000,92.400000,3.100000,7.600000
33,21705,FINALIZACION,70.325000,95.650000,3.195000,4.750000
34,21706,FINALIZACION,70.615000,94.000000,3.150000,6.000000


In [35]:
qa_control_comp = qa_control_comp.drop_duplicates(subset=['op'], keep="first")
df = pd.merge(df, qa_agro_comp, how='left', on='op')
df = pd.merge(df, qa_control_comp, how='left', on='op')

In [36]:
q_min = df["sackoff_op"].quantile(0.01)
q_max = df["sackoff_op"].quantile(0.99)

cond1 = df["sackoff_op"] >= q_min
cond2 = df["sackoff_op"] <= q_max
cond_range = cond1 & cond2

# registro que quieres mandar a df_bad
cond_op_21944 = df["op"].isin(["21944","21864"])

# filas “buenas”: dentro del rango Y NO op 21944
conds = cond_range & ~cond_op_21944

df_cut = df[conds].copy()
df_bad = df[~conds].copy()



df_group_diet = compute_sackoff(df_cut, cols=["Dieta"])
mask = (df_group_diet
        .groupby('Dieta')['Tiene Adiflow']
        .transform('nunique')
        .eq(2))

df_both_diet = (df_group_diet[mask]
           .sort_values(['Dieta', 'Tiene Adiflow']))["Dieta"].unique()

df_both_cut = df_cut[df_cut["Dieta"].isin(df_both_diet)]

In [37]:
df_bad_dep = df_bad[['date',
        'op',
        'Dieta',
        'panificadas',
       'entregadas',
        'Producción (Ton)',
        'Anulación (Ton)',
       #'peso_real',
        'diff', 'sackoff_op',
        'Tiene Adiflow',
        'pdi_agro',
       'dureza_agro',
        'finos_agro']].round(2)

df_bad_dep = df_bad_dep.rename(columns={'diff': 'Diferencia (Ton)',
                                        'panificadas': "Planificadas (Ton)",
                                        "entregadas": "Entregadas (Ton)",

                                        'sackoff_op': 'Sackoff (%)',
                           'pdi_agro': 'Pdi (%)', 'dureza_agro': 'Dureza (kg/cm2)', 'finos_agro': 'Finos (%)'})

df_bad_dep = df_bad_dep.sort_values(["date"], ascending=False)
s3.save_dataframe(df_bad_dep, "data_bad.csv")
df_bad_dep

,date,op,Dieta,Planificadas (Ton),Entregadas (Ton),Producción (Ton),Anulación (Ton),Diferencia (Ton),Sackoff (%),Tiene Adiflow,Pdi (%),Dureza (kg/cm2),Finos (%)
468,2025-12-23,22091,MACHOS ESP 113,2.3,2.30,2.01,0.00,0.29,14.38,Sin Adiflow,90.80,3.00,9.20
349,2025-11-15,21944,GESTACION ESP P,28.0,23.66,28.05,0.00,-4.39,-15.65,Con Adiflow,89.80,3.10,10.20
285,2025-11-01,21864,FLUSHING,42.0,37.92,42.09,0.00,-4.17,-9.90,Con Adiflow,94.25,3.03,5.75
231,2025-10-20,21796,GESTACION,10.0,8.07,10.03,0.00,-1.96,-19.53,Con Adiflow,91.40,3.00,8.60
208,2025-10-14,21747,LEVANTE CMC,26.0,29.26,26.04,0.00,3.22,12.39,Sin Adiflow,93.40,3.00,6.60
190,2025-10-11,21759,MACHOS ESP 113,2.0,1.33,1.99,0.00,-0.66,-33.24,Con Adiflow,90.20,3.00,9.80
179,2025-10-09,21752,REEMPLAZOS S1B,12.0,12.84,12.06,0.00,0.77,6.39,Sin Adiflow,93.30,3.60,6.70
174,2025-10-08,21739,LEVANTE CMC,26.0,26.00,26.08,16.05,4.12,15.80,Sin Adiflow,90.95,3.05,9.05
164,2025-10-06,21732,NOVILLONAS SUPLEMENTO,4.0,4.41,4.00,0.00,0.41,10.17,Con Adiflow,97.20,3.60,2.80
157,2025-10-05,21722,REEMPLAZOS S1B,26.0,26.52,24.94,0.00,1.57,6.31,Con Adiflow,90.80,3.21,9.20


In [38]:
sackoff_date = compute_sackoff(df_both_cut, cols=["date"])

In [39]:
for con in sackoff_date["Tiene Adiflow"].unique():
    f = plot_dynamic_trends(
        df=sackoff_date[sackoff_date["Tiene Adiflow"] == con], 
        date_col="date", 
        value_col="sackoff",
        category_col="Tiene Adiflow", 
            windows=[7, 15, 20],
        width=1200, 
        height=400  
    )
    f.show()
    name = f"fazenda_sackoff_{con.replace(' ', '_').lower()}.html"
    print(name)
    s3.save_plotly_html(f, name)

fazenda_sackoff_con_adiflow.html


fazenda_sackoff_sin_adiflow.html


In [40]:


def plot_multi_window_trends_styled(
    df: pd.DataFrame,
    date_col: str,
    value_col: str,
    category_col: str,
    windows: list = [7, 15, 20],
    colors: dict = {"Con Adiflow": "#2E86C1", "Sin Adiflow": "#E74C3C"},
    title: str = "Comparativa de Tendencias Multi-Ventana",
    height: int = 600,
    width: int = 1200
) -> go.Figure:
    """
    Grafica múltiples medias móviles con estilos mejorados:
    - Incluye puntos (markers) en los datos.
    - Líneas más gruesas.
    - Títulos, etiquetas y marcos de ejes en color negro.
    """
    
    d = df.copy()
    d[date_col] = pd.to_datetime(d[date_col])
    d = d.sort_values(by=[date_col])
    
    fig = go.Figure()
    
    line_styles = ["dot", "dash", "solid"] 
    windows_sorted = sorted(windows)
    style_map = {w: line_styles[i % len(line_styles)] for i, w in enumerate(windows_sorted)}

    unique_cats = d[category_col].unique()

    for cat in unique_cats:
        subset = d[d[category_col] == cat].copy()
        base_color = colors.get(cat, "#333333")
        
        for w in windows_sorted:
            col_name = f"MA_{w}"
            # Calculamos la media móvil
            subset[col_name] = subset[value_col].rolling(window=w, min_periods=1).mean()
            
            trace_name = f"{cat} ({w} días)"
            
            # Determinamos el grosor de la línea (AUMENTADO)
            # La ventana más grande es la más gruesa (3), las otras un poco menos (2.5)
            line_width = 3 if w == max(windows) else 2.5
            
            fig.add_trace(
                go.Scatter(
                    x=subset[date_col],
                    y=subset[col_name],
                    mode='lines+markers', # --- CAMBIO 1: Añadido markers ---
                    name=trace_name,
                    line=dict(
                        color=base_color, 
                        width=line_width, # --- CAMBIO 2: Grosor aumentado ---
                        dash=style_map[w]
                    ),
                    marker=dict( # Configuración de los puntos
                        size=6,
                        color=base_color,
                        symbol='circle'
                    ),
                    hovertemplate=(
                        f"<b>{cat}</b><br>"
                        f"Ventana: {w} días<br>"
                        f"Fecha: %{{x}}<br>"
                        f"Valor: %{{y:.2f}}<extra></extra>"
                    )
                )
            )

    # --- CAMBIO 3: Configuración de estilos en Negro ---
    fig.update_layout(
        title={'text': f"<b>{title}</b>", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
        # Aseguramos que la fuente global sea negra
        font=dict(family="Inter, Arial, sans-serif", size=12, color="black"),
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=60, r=40, t=80, b=60), # Márgenes ajustados ligeramente para los ejes más gruesos
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        height=height,
        width=width,
        hovermode="x unified"
    )
    
    # Configuración de Ejes
    axis_style = dict(
        showline=True,     
        linecolor="black", 
        ticks="outside",   
        tickcolor="black",
        tickfont=dict(color="black"),             
        title_font=dict(color="black", size=14),  
        showgrid=True,
        gridcolor='rgba(0,0,0,0.1)' 
    )

    fig.update_xaxes(title_text="Fecha", **axis_style)
    fig.update_yaxes(
        title_text=value_col, 
        zeroline=True, 
        zerolinecolor="black",
        zerolinewidth=2,
        **axis_style
    )

    return fig

In [41]:
# Definimos colores personalizados si quieres (opcional)
mis_colores = {
    "Con Adiflow": "#1C8074",  
    "Sin Adiflow": "#666666" 
}

fig = plot_multi_window_trends_styled(
    df=sackoff_date,
    date_col="date",
    value_col="sackoff",
    category_col="Tiene Adiflow",
    windows=[7, 15, 30],
    colors=mis_colores
)

fig.show()
s3.save_plotly_html(fig, "fazenda_efecto_adiflow_en_sackoff_tendencia_comparativa.html")

In [42]:
by_month = compute_sackoff(df_both_cut, cols=["month"])
by_month_sackoff = by_month.groupby("Tiene Adiflow").agg(
    sackoff_mean=('sackoff_mean', 'mean'),
    sackoff=('sackoff', 'mean'),
).reset_index()
con_adiflow = by_month_sackoff.loc[by_month_sackoff['Tiene Adiflow']=='Con Adiflow', "sackoff"][0]
sin_adiflow =  by_month_sackoff.loc[by_month_sackoff['Tiene Adiflow']!='Con Adiflow', "sackoff"][1]

In [43]:
summary_general_cut = compute_sackoff(df_cut, cols=[])
cols_visual = [
    'Tiene Adiflow',
    'ops',
    'production',
    'reales',
    'anulation',
    'sackoff',
    'diferencia',
    'temp1_acond_c',
    'pdi',
    'finos',
    'dureza']
for col in cols_visual:
    if col !='Tiene Adiflow':
        summary_general_cut[col] = pd.to_numeric(summary_general_cut[col], errors='coerce')
summary_general_cut = summary_general_cut[cols_visual].round(2)
summary_general_cut = summary_general_cut.rename(columns={
    'ops': 'OPs',
    'production': 'Planificadas (Ton)',
     'reales': 'Producidas (Ton)',
    'anulation': 'Anuladas (Ton)',
    'sackoff': 'Sackoff Prom (%)',
    'diferencia': "Diferencia (Ton)",
    'temp1_acond_c': 'Temp (°C)',
    'pdi': 'Pdi (%)',
    'finos': 'Finos (%)',
    'dureza': 'Dureza (Kg/cm2)',
})
#summary_general_cut.loc[summary_general_cut["Tiene Adiflow"]=="Con Adiflow", "Sackoff Prom (%)"] = con_adiflow
#summary_general_cut.loc[summary_general_cut["Tiene Adiflow"]=="Sin Adiflow", "Sackoff Prom (%)"] = sin_adiflow
s3.save_dataframe(summary_general_cut, "summary_general.csv")
summary_general_cut

,Tiene Adiflow,OPs,Planificadas (Ton),Producidas (Ton),Anuladas (Ton),Sackoff Prom (%),Diferencia (Ton),Temp (°C),Pdi (%),Finos (%),Dureza (Kg/cm2)
0,Con Adiflow,290,22509.11,22781.35,43.83,-0.58,-131.68,74.69,93.33,6.65,3.11
1,Sin Adiflow,237,12447.34,12437.34,63.21,-0.85,-105.61,72.23,93.91,6.10,3.10


In [44]:
def extract_svg_params(df: pd.DataFrame, fecha_ini: str, fecha_fin: str, pct_datos: int) -> dict:
    """
    Convierte el DataFrame de resumen en un diccionario plano de parámetros 
    para la función de generación de SVG.
    """
    # 1. Convertir a diccionario anidado usando 'Tiene Adiflow' como clave
    # Resultado: {'Con Adiflow': {'Producidas (Ton)': 2000, ...}, 'Sin Adiflow': {...}}
    data = df.set_index('Tiene Adiflow').to_dict(orient='index')
    con = data.get('Con Adiflow', {})
    sin = data.get('Sin Adiflow', {})
    ton_con_adiflow = con.get('Producidas (Ton)', 0)
    ton_sin_adiflow = sin.get('Producidas (Ton)', 0)
    sackoff_con_adiflow = con.get('Sackoff Prom (%)', 0)
    sackoff_sin_adiflow = sin.get('Sackoff Prom (%)', 0)
    mejora_pct = sackoff_con_adiflow-sackoff_sin_adiflow
    recuperadas_prom = mejora_pct*ton_con_adiflow/100
   
    
    return {
        # Toneladas
        "ton_con_adiflow": ton_con_adiflow,
        "ton_sin_adiflow": ton_sin_adiflow,

        "mejora_pct": mejora_pct, 
        "sackoff_con": sackoff_con_adiflow,
        "sackoff_sin": sackoff_sin_adiflow,
        "recuperadas_prom": recuperadas_prom, 
        
        # Temperatura
        "temp_con": con.get('Temp (°C)', 0),
        "temp_sin": sin.get('Temp (°C)', 0),
        "delta_temp": con.get('Temp (°C)', 0) - sin.get('Temp (°C)', 0),
        
        # Calidad
        "pdi_con": con.get('Pdi (%)', 0),
        "pdi_sin": sin.get('Pdi (%)', 0),
        "finos_con": con.get('Finos (%)', 0),
        "finos_sin": sin.get('Finos (%)', 0),
        
        # Metadatos externos (pasados como argumentos)
        "fecha_ini": fecha_ini,
        "fecha_fin": fecha_fin,
        "pct_datos": pct_datos,
    }

In [45]:
svg_params = extract_svg_params(
    df=summary_general_cut,
    fecha_ini=df_cut["date"].min().strftime("%d-%b"),  
    fecha_fin=df_cut["date"].max().strftime("%d-%b"),
    pct_datos=98
)
svg_params

{'ton_con_adiflow': 22781.35,
 'ton_sin_adiflow': 12437.34,
 'mejora_pct': 0.27,
 'sackoff_con': -0.58,
 'sackoff_sin': -0.85,
 'recuperadas_prom': 61.509645,
 'temp_con': 74.69,
 'temp_sin': 72.23,
 'delta_temp': 2.4599999999999937,
 'pdi_con': 93.33,
 'pdi_sin': 93.91,
 'finos_con': 6.65,
 'finos_sin': 6.1,
 'fecha_ini': '01-Sep',
 'fecha_fin': '30-Dec',
 'pct_datos': 98}

In [46]:

s3_template_url = f"s3://{s3.bucket_name}/svg_template/fazenda_sackoff.svg"
output_filename = "fazenda_sackoff.svg"


svg_params = extract_svg_params(
    df=summary_general_cut,
    fecha_ini=df_cut["date"].min().strftime("%d-%b"),  
    fecha_fin=df_cut["date"].max().strftime("%d-%b"),
    pct_datos=98
)
svg_content = generate_sackoff_svg(
    template_path=s3_template_url, 
    **svg_params
)
s3.save_svg_content(svg_content, output_filename)


's3://galileo-c4e9a2f1/images/fazenda_efecto_adiflow_en_sackoff/fazenda_sackoff.svg'

In [47]:
compute_sackoff(df_cut, cols=[])

,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,Con Adiflow,-131.679961,22781.352617,22509.112961,43.829,-0.285235,290,74.687080,93.332457,94.805681,6.648562,6.573963,3.108978,3.169025,-0.578016
1,Sin Adiflow,-105.605600,12437.337600,12447.337600,63.210,-0.588466,237,72.229365,93.913148,94.032346,6.096111,5.964502,3.102963,3.609426,-0.849101


In [48]:
summary_table = build_summary_table(summary_general_cut)
s3.save_dataframe(summary_table, "summary_recuperadas.csv")
summary_table

,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas
0,-0.85,-0.58,0.27,22781.35,61.51


In [49]:
cols = ['diff', 'sackoff_op']
group_cols = ["month",'Tiene Adiflow']
# Cantidad de registros por grupo
n_por_grupo = (
    df.groupby(group_cols)
      .size()
      .rename('n_registros')
)

# Describe solo para diff y sackoff_op
stats = (
    df
    .groupby(group_cols)[cols]
    .describe(percentiles=[0.25, 0.5, 0.75])
)

# Aplanar columnas
stats.columns = [f"{col}_{stat}" for col, stat in stats.columns]

# Unir con n_registros
resumen = stats.join(n_por_grupo)
resumen

diff_count  diff_mean  diff_std   diff_min  diff_25%  \
month   Tiene Adiflow                                                         
2025-09 Con Adiflow          52.0  -0.102762  1.251226  -3.672686 -0.482699   
        Sin Adiflow          81.0  -0.596387  2.650729 -17.465216 -0.417000   
2025-10 Con Adiflow         115.0  -0.481271  1.542875  -6.049869 -0.939699   
        Sin Adiflow          36.0  -0.151081  1.298330  -4.762731 -0.538959   
2025-11 Con Adiflow          84.0  -0.667250  1.384925  -4.425972 -1.300815   
        Sin Adiflow          37.0  -0.276430  0.632888  -1.727688 -0.530502   
2025-12 Con Adiflow          46.0  -0.524797  1.334429  -6.918253 -1.006520   
        Sin Adiflow          91.0  -0.693004  1.359804  -5.338525 -1.119000   

                       diff_50%  diff_75%  diff_max  sackoff_op_count  \
month   Tiene Adiflow                                                   
2025-09 Con Adiflow    0.001250  0.659363  2.366365              52.0   
        Sin Adiflow   -0.011622  0.416220  2.976560              81.0   
2025-10 Con Adiflow   -0.163766  0.159974  5.358398             115.0   
        Sin Adiflow   -0.102718  0.074235  4.120487              36.0   
2025-11 Con Adiflow   -0.415562  0.028875  3.367139              84.0   
        Sin Adiflow   -0.093668  0.157732  0.709246              37.0   
2025-12 Con Adiflow   -0.146795  0.016487  3.819509              46.0   
        Sin Adiflow   -0.449832  0.016184  5.580049              91.0   

                       sackoff_op_mean  sackoff_op_std  sackoff_op_min  \
month   Tiene Adiflow                                                    
2025-09 Con Adiflow          -0.209983        1.930445       -5.802248   
        Sin Adiflow          -2.257577        5.656380      -24.706820   
2025-10 Con Adiflow          -1.085768        4.518162      -33.236116   
        Sin Adiflow          -0.895480        5.483696      -16.971731   
2025-11 Con Adiflow          -0.688760        2.538344      -15.652665   
        Sin Adiflow          -0.945399        2.429844       -9.123147   
2025-12 Con Adiflow          -0.920660        2.650400      -10.761637   
        Sin Adiflow          -1.207996        3.171111      -14.342308   

                       sackoff_op_25%  sackoff_op_50%  sackoff_op_75%  \
month   Tiene Adiflow                                                   
2025-09 Con Adiflow         -0.530970        0.003046        0.700463   
        Sin Adiflow         -2.845568       -0.037205        0.736867   
2025-10 Con Adiflow         -1.579086       -0.232223        0.638541   
        Sin Adiflow         -2.557485       -0.342737        0.782212   
2025-11 Con Adiflow         -1.359271       -0.519343        0.652388   
        Sin Adiflow         -1.870588       -0.334009        0.720015   
2025-12 Con Adiflow         -1.154116       -0.387554        0.255145   
        Sin Adiflow         -2.039703       -0.966066        0.213598   

                       sackoff_op_max  n_registros  
month   Tiene Adiflow                               
2025-09 Con Adiflow          4.610601           52  
        Sin Adiflow          5.268541           81  
2025-10 Con Adiflow         10.165998          115  
        Sin Adiflow         15.799709           36  
2025-11 Con Adiflow          6.243745           84  
        Sin Adiflow          2.562822           37  
2025-12 Con Adiflow          5.538853           46  
        Sin Adiflow         14.380390           91

In [50]:

# Estadísticos por grupo (para todas las columnas numéricas)
stats = (
    df
    .groupby('Tiene Adiflow')
    .describe(percentiles=[0.25, 0.5, 0.75])
)

# Aplanar columnas tipo (columna, métrica) -> columna_métrica
stats.columns = [f"{col}_{stat}" for col, stat in stats.columns]
stats

,order_count,order_mean,order_std,order_min,order_25%,order_50%,order_75%,order_max,panificadas_count,panificadas_mean,...,dureza_75%,dureza_max,finos_count,finos_mean,finos_std,finos_min,finos_25%,finos_50%,finos_75%,finos_max
Tiene Adiflow,,,,,,,,,,,,,,,,,,,,,
Con Adiflow,297.0,1.001553e+07,309.331116,10014857.0,10015292.0,10015521.0,10015765.0,10016141.0,297.0,77.189226,...,3.2,3.5,147.0,6.666618,2.193789,1.0,5.333333,6.8,8.05,14.9
Sin Adiflow,245.0,1.001559e+07,448.792596,10014855.0,10015136.0,10015620.0,10016015.0,10016241.0,245.0,52.128571,...,3.2,4.3,65.0,6.068538,2.918541,0.3,4.600000,6.4,8.00,10.3


In [51]:
df_cut_group_month = compute_sackoff(df_cut, cols=["month"])        # lista ok
df_cut_group_month

,month,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,2025-09,Con Adiflow,-5.343609,4277.010709,4240.599609,0.000,0.003046,52,74.303205,91.937821,100.166426,8.008333,7.779851,3.059295,3.071147,-0.124938
1,2025-09,Sin Adiflow,-18.469391,4148.530391,4158.530391,63.210,0.000000,77,69.082609,92.278261,93.623564,7.762319,6.286051,3.021159,4.464974,-0.445203
2,2025-10,Con Adiflow,-54.706258,8373.738508,8319.481258,36.952,-0.232223,111,74.622082,93.515467,93.611565,6.492866,6.355360,3.149664,3.234853,-0.653307
3,2025-10,Sin Adiflow,-13.555748,920.843748,920.843748,0.000,-0.512103,33,72.550000,94.291667,93.749000,5.708333,6.251000,3.223333,3.071333,-1.472101
4,2025-11,Con Adiflow,-47.489442,7158.510398,6998.702442,0.000,-0.476465,81,74.885522,93.734353,93.676183,6.239403,6.335182,3.097289,3.133742,-0.663398
5,2025-11,Sin Adiflow,-10.227900,1501.350900,1501.350900,0.000,-0.334009,37,75.137500,95.048056,94.894103,4.937361,5.138462,3.110417,3.113462,-0.681246
6,2025-12,Con Adiflow,-24.140652,2972.093002,2950.329652,6.877,-0.387554,46,NaN,NaN,93.812417,NaN,6.191542,NaN,3.184208,-0.812244
7,2025-12,Sin Adiflow,-63.352561,5866.612561,5866.612561,0.000,-0.982825,90,70.000000,98.600000,94.216753,1.400000,5.856534,3.000000,3.129699,-1.079883


In [106]:
cols_visual = [
    'month',
    'Tiene Adiflow',
    'ops',
    'production',
    'reales',
    'anulation',
    'sackoff',
    'diferencia',
    'temp1_acond_c',
    'pdi',
    'finos',
    'dureza']
summary_month = df_cut_group_month.copy()
for col in cols_visual:
    if col not in ['Tiene Adiflow', 'month']:
        summary_month[col] = pd.to_numeric(summary_month[col], errors='coerce')
summary_month = summary_month[cols_visual].round(2)
summary_month = summary_month.rename(columns={
    'ops': 'OPs',
    'production': 'Planificadas (Ton)',
     'reales': 'Producidas (Ton)',
    'anulation': 'Anuladas (Ton)',
    'sackoff': 'Sackoff (%)',
    'diferencia': "Diferencia (Ton)",
    'temp1_acond_c': 'Temp (°C)',
    'pdi': 'Pdi (%)',
    'finos': 'Finos (%)',
    'dureza': 'Dureza (Kg/cm2)',
})
s3.save_dataframe(summary_month, 'summary_month.csv')
summary_month

,month,Tiene Adiflow,OPs,Planificadas (Ton),Producidas (Ton),Anuladas (Ton),Sackoff (%),Diferencia (Ton),Temp (°C),Pdi (%),Finos (%),Dureza (Kg/cm2)
0,2025-09,Con Adiflow,52,4240.60,4277.01,0.00,-0.12,-5.34,74.30,91.94,8.01,3.06
1,2025-09,Sin Adiflow,77,4158.53,4148.53,63.21,-0.45,-18.47,69.08,92.28,7.76,3.02
2,2025-10,Con Adiflow,111,8319.48,8373.74,36.95,-0.65,-54.71,74.62,93.52,6.49,3.15
3,2025-10,Sin Adiflow,33,920.84,920.84,0.00,-1.47,-13.56,72.55,94.29,5.71,3.22
4,2025-11,Con Adiflow,81,6998.70,7158.51,0.00,-0.66,-47.49,74.89,93.73,6.24,3.10
5,2025-11,Sin Adiflow,37,1501.35,1501.35,0.00,-0.68,-10.23,75.14,95.05,4.94,3.11
6,2025-12,Con Adiflow,46,2950.33,2972.09,6.88,-0.81,-24.14,NaN,NaN,NaN,NaN
7,2025-12,Sin Adiflow,90,5866.61,5866.61,0.00,-1.08,-63.35,70.00,98.60,1.40,3.00


In [107]:
df_cut_group_month

,month,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,2025-09,Con Adiflow,-5.343609,4277.010709,4240.599609,0.000,0.003046,52,74.303205,91.937821,100.166426,8.008333,7.779851,3.059295,3.071147,-0.124938
1,2025-09,Sin Adiflow,-18.469391,4148.530391,4158.530391,63.210,0.000000,77,69.082609,92.278261,93.623564,7.762319,6.286051,3.021159,4.464974,-0.445203
2,2025-10,Con Adiflow,-54.706258,8373.738508,8319.481258,36.952,-0.232223,111,74.622082,93.515467,93.611565,6.492866,6.355360,3.149664,3.234853,-0.653307
3,2025-10,Sin Adiflow,-13.555748,920.843748,920.843748,0.000,-0.512103,33,72.550000,94.291667,93.749000,5.708333,6.251000,3.223333,3.071333,-1.472101
4,2025-11,Con Adiflow,-47.489442,7158.510398,6998.702442,0.000,-0.476465,81,74.885522,93.734353,93.676183,6.239403,6.335182,3.097289,3.133742,-0.663398
5,2025-11,Sin Adiflow,-10.227900,1501.350900,1501.350900,0.000,-0.334009,37,75.137500,95.048056,94.894103,4.937361,5.138462,3.110417,3.113462,-0.681246
6,2025-12,Con Adiflow,-24.140652,2972.093002,2950.329652,6.877,-0.387554,46,NaN,NaN,93.812417,NaN,6.191542,NaN,3.184208,-0.812244
7,2025-12,Sin Adiflow,-63.352561,5866.612561,5866.612561,0.000,-0.982825,90,70.000000,98.600000,94.216753,1.400000,5.856534,3.000000,3.129699,-1.079883


In [108]:
summary_month_table = build_monthly_summary_table(
    summary_month,
    month_order=sorted(summary_month['month'].unique()),
)
s3.save_dataframe(summary_month_table, 'summary_month_table.csv')
summary_month_table

,Mes,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas
0,2025-09,-0.45,-0.12,0.33,4277.01,14.11
1,2025-10,-1.47,-0.65,0.82,8373.74,68.66
2,2025-11,-0.68,-0.66,0.02,7158.51,1.43
3,2025-12,-1.08,-0.81,0.27,2972.09,8.02


In [109]:
m = df_cut_group_month["month"].dt.to_timestamp()
df_cut_group_month["month_lbl"] = m.dt.strftime("%b-%Y")
cats = pd.date_range(m.min(), m.max(), freq="MS").strftime("%b-%Y")
df_cut_group_month["month_lbl"] = pd.Categorical(df_cut_group_month["month_lbl"], categories=cats, ordered=True)
df_cut_group_month

,month,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff,month_lbl
0,2025-09,Con Adiflow,-5.343609,4277.010709,4240.599609,0.000,0.003046,52,74.303205,91.937821,100.166426,8.008333,7.779851,3.059295,3.071147,-0.124938,Sep-2025
1,2025-09,Sin Adiflow,-18.469391,4148.530391,4158.530391,63.210,0.000000,77,69.082609,92.278261,93.623564,7.762319,6.286051,3.021159,4.464974,-0.445203,Sep-2025
2,2025-10,Con Adiflow,-54.706258,8373.738508,8319.481258,36.952,-0.232223,111,74.622082,93.515467,93.611565,6.492866,6.355360,3.149664,3.234853,-0.653307,Oct-2025
3,2025-10,Sin Adiflow,-13.555748,920.843748,920.843748,0.000,-0.512103,33,72.550000,94.291667,93.749000,5.708333,6.251000,3.223333,3.071333,-1.472101,Oct-2025
4,2025-11,Con Adiflow,-47.489442,7158.510398,6998.702442,0.000,-0.476465,81,74.885522,93.734353,93.676183,6.239403,6.335182,3.097289,3.133742,-0.663398,Nov-2025
5,2025-11,Sin Adiflow,-10.227900,1501.350900,1501.350900,0.000,-0.334009,37,75.137500,95.048056,94.894103,4.937361,5.138462,3.110417,3.113462,-0.681246,Nov-2025
6,2025-12,Con Adiflow,-24.140652,2972.093002,2950.329652,6.877,-0.387554,46,NaN,NaN,93.812417,NaN,6.191542,NaN,3.184208,-0.812244,Dec-2025
7,2025-12,Sin Adiflow,-63.352561,5866.612561,5866.612561,0.000,-0.982825,90,70.000000,98.600000,94.216753,1.400000,5.856534,3.000000,3.129699,-1.079883,Dec-2025


In [110]:
f = plot_bar(
    df_cut_group_month.round(2),
    x_col="month_lbl",
    y_col="sackoff",
    group_col="Tiene Adiflow",
    order_x=sorted(df_cut_group_month["month_lbl"].unique(), reverse=True),
    title="Sackoff Total por mes",
    cat_base="Sin Adiflow",
    show_delta=True,
    x_title="Mes",
    y_title="Sackoff",
    text_format=".2f",
    delta_unit="%",
    hover_data_cols=['reales', 'production'],
    height=500,
    width=1000,
)
f.show()
s3.save_plotly_html(f, "barras_sackoff_mes.html")

In [111]:
df_group_dieta_ = compute_sackoff(df_both_cut, cols=["Dieta"])        # lista ok
df_group_dieta_

,Dieta,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,ENGORDE ESP P,Con Adiflow,-0.229702,1083.730002,1073.833702,0.000,-0.418632,11,75.625000,93.087500,93.425000,6.970833,6.571296,3.084167,3.105000,-0.021195
1,ENGORDE ESP P,Sin Adiflow,-7.622713,721.934713,721.934713,27.520,-0.971161,8,73.475000,92.125000,93.535714,7.875000,6.450000,3.087500,3.192857,-1.055873
2,FINALIZACION,Con Adiflow,-19.406636,3186.224636,3160.980636,15.250,-0.298929,34,75.275500,93.423333,93.711979,6.545417,6.313906,3.156667,3.176354,-0.609079
3,FINALIZACION,Sin Adiflow,-3.227420,240.509420,240.509420,0.000,-1.266560,2,NaN,NaN,94.000000,NaN,6.000000,NaN,3.100000,-1.341910
4,FLUSHING,Con Adiflow,-3.422692,395.100292,392.737692,0.000,0.008829,13,70.933333,95.680000,95.300000,4.320000,4.700000,3.146000,3.155667,-0.866284
5,FLUSHING,Sin Adiflow,-3.333668,357.297668,357.297668,0.000,-1.380618,10,74.000000,93.566667,95.520833,6.433333,4.479167,3.133333,3.127083,-0.933023
6,GESTACION,Con Adiflow,-16.788823,3719.805673,3694.035823,0.000,-0.171114,32,77.679273,91.087218,91.048246,8.882957,8.870196,3.122607,3.136733,-0.451336
7,GESTACION,Sin Adiflow,-14.923741,1033.793741,1033.793741,0.000,-1.175544,10,80.000000,90.000000,91.351667,10.000000,8.651333,3.000000,3.177500,-1.443590
8,GESTACION ESP P,Con Adiflow,-16.378488,1483.471388,1473.927488,21.702,-0.760457,13,78.508333,90.283333,90.641490,9.768056,9.362100,3.081111,3.127430,-1.104065
9,GESTACION ESP P,Sin Adiflow,-4.257816,104.351816,104.351816,0.000,-3.855748,3,NaN,NaN,91.300000,NaN,8.700000,NaN,3.200000,-4.080251


In [112]:
# Partimos de df_both (ya solo dietas con ambos estados)
pivot = (
    df_group_dieta_
    .pivot_table(
        index="Dieta",
        columns="Tiene Adiflow",
        values="sackoff",
        aggfunc="mean"   # o 'sum' según cómo definas
    )
)

# Dietas donde CON Adiflow tiene mejor sackoff (= valor más alto, menos pérdida)
df_mejor_con = (
    pivot[pivot["Con Adiflow"] > pivot["Sin Adiflow"]]
    .assign(delta=lambda x: x["Con Adiflow"] - x["Sin Adiflow"])
    .sort_values("delta", ascending=False)
    .reset_index()
)

df_mejor_con = df_mejor_con.round(2)
s3.save_dataframe(df_mejor_con, "mejores_resultados.csv")
df_mejor_con

Tiene Adiflow,Dieta,Con Adiflow,Sin Adiflow,delta
0,MACHOS ESP 113,1.02,-2.05,3.08
1,GESTACION ESP P,-1.10,-4.08,2.98
2,NOVILLONAS SUPLEMENTO,-0.30,-2.73,2.42
3,REEMPLAZOS S1B,0.47,-1.65,2.12
4,PRELACTANCIA H,0.53,-1.50,2.03
5,LACTANCIA PRIMERIZAS ESP,-0.67,-2.38,1.71
6,LEVANTE CMC,-0.40,-1.69,1.29
7,PREINICIACION F2 P INMUNIDAD,1.30,0.09,1.21
8,ENGORDE ESP P,-0.02,-1.06,1.03
9,GESTACION,-0.45,-1.44,0.99


In [113]:
cols_visual = [
    'Dieta',
    'Tiene Adiflow',
    'ops',
    'production',
    'reales',
    'anulation',
    'sackoff',
    'diferencia',
    'temp1_acond_c',
    'pdi',
    'finos',
    'dureza']
summary_dieta = df_group_dieta_.copy()
for col in cols_visual:
    if col not in ['Tiene Adiflow', 'Dieta']:
        summary_dieta[col] = pd.to_numeric(summary_dieta[col], errors='coerce')
summary_dieta = summary_dieta[cols_visual].round(2)
summary_dieta = summary_dieta.rename(columns={
    'ops': 'OPs',
    'production': 'Planificadas (Ton)',
     'reales': 'Producidas (Ton)',
    'anulation': 'Anuladas (Ton)',
    'sackoff': 'Sackoff (%)',
    'diferencia': "Diferencia (Ton)",
    'temp1_acond_c': 'Temp (°C)',
    'pdi': 'Pdi (%)',
    'finos': 'Finos (%)',
    'dureza': 'Dureza (Kg/cm2)',
})
summary_dieta = summary_dieta[summary_dieta["Dieta"].isin(df_mejor_con["Dieta"])]
s3.save_dataframe(summary_dieta, "summary_dieta.csv")
summary_dieta

,Dieta,Tiene Adiflow,OPs,Planificadas (Ton),Producidas (Ton),Anuladas (Ton),Sackoff (%),Diferencia (Ton),Temp (°C),Pdi (%),Finos (%),Dureza (Kg/cm2)
0,ENGORDE ESP P,Con Adiflow,11,1073.83,1083.73,0.00,-0.02,-0.23,75.62,93.09,6.97,3.08
1,ENGORDE ESP P,Sin Adiflow,8,721.93,721.93,27.52,-1.06,-7.62,73.47,92.12,7.88,3.09
2,FINALIZACION,Con Adiflow,34,3160.98,3186.22,15.25,-0.61,-19.41,75.28,93.42,6.55,3.16
3,FINALIZACION,Sin Adiflow,2,240.51,240.51,0.00,-1.34,-3.23,NaN,NaN,NaN,NaN
4,FLUSHING,Con Adiflow,13,392.74,395.10,0.00,-0.87,-3.42,70.93,95.68,4.32,3.15
5,FLUSHING,Sin Adiflow,10,357.30,357.30,0.00,-0.93,-3.33,74.00,93.57,6.43,3.13
6,GESTACION,Con Adiflow,32,3694.04,3719.81,0.00,-0.45,-16.79,77.68,91.09,8.88,3.12
7,GESTACION,Sin Adiflow,10,1033.79,1033.79,0.00,-1.44,-14.92,80.00,90.00,10.00,3.00
8,GESTACION ESP P,Con Adiflow,13,1473.93,1483.47,21.70,-1.10,-16.38,78.51,90.28,9.77,3.08
9,GESTACION ESP P,Sin Adiflow,3,104.35,104.35,0.00,-4.08,-4.26,NaN,NaN,NaN,NaN


In [114]:
summary_month_table = build_monthly_summary_table(
    summary_dieta,
     month_col= "Dieta",
    month_order=df_mejor_con["Dieta"].unique().tolist(),
)
summary_month_table.rename(columns={"Mes": "Dieta"}, inplace=True)
summary_month_table = summary_month_table.sort_values("Toneladas Recuperadas", ascending=False)
s3.save_dataframe(summary_month_table, f"summary_dieta_table.csv")
summary_month_table

,Dieta,Sackoff Sin Adiflow,Sackoff Con Adiflow,Diferencia Sackoff,Toneladas Producidas Con Adiflow,Toneladas Recuperadas
1,GESTACION ESP P,-4.08,-1.10,2.98,1483.47,44.21
9,GESTACION,-1.44,-0.45,0.99,3719.81,36.83
10,FINALIZACION,-1.34,-0.61,0.73,3186.22,23.26
5,LACTANCIA PRIMERIZAS ESP,-2.38,-0.67,1.71,1110.63,18.99
14,LEVANTE,-0.65,-0.27,0.38,3092.58,11.75
8,ENGORDE ESP P,-1.06,-0.02,1.04,1083.73,11.27
7,PREINICIACION F2 P INMUNIDAD,0.09,1.30,1.21,462.38,5.59
15,LEVANTE ESP P,-0.28,-0.05,0.23,2182.45,5.02
6,LEVANTE CMC,-1.69,-0.40,1.29,257.69,3.32
11,LACTANCIA SILO H,-0.89,-0.24,0.65,404.66,2.63


In [115]:
f = plot_bar(
    df_group_dieta_.round(2),
    x_col="Dieta",
    y_col="sackoff",
    group_col="Tiene Adiflow",
    order_x=["Septiembre", "Octubre", "Noviembre", "Diciembre"],
    title="Sackoff Total por Dieta",
    cat_base="Sin Adiflow",
    show_delta=True,
    x_title="Mes",
    y_title="Sackoff",
    text_format=".2f",
    delta_unit="%",
    hover_data_cols=['reales', 'production'],
    height=500,
    width=1000,
)
f.show()
s3.save_plotly_html(f, "barras_sackoff_dieta_both.html")

In [117]:
df_group_dieta_mes = compute_sackoff(df_both_cut, cols=["Dieta", "month"])

In [118]:
months = pd.PeriodIndex(["2025-11", "2025-12"], freq="M")
df_cut_nov = df_cut[df_cut["month"].isin(months)]
compute_sackoff(df_cut_nov, cols=[])

,Tiene Adiflow,diferencia,reales,production,anulation,sackoff_mean,ops,temp1_acond_c,pdi,pdi_agro,finos,finos_agro,dureza,dureza_agro,sackoff
0,Con Adiflow,-71.630094,10130.603400,9949.032094,6.877,-0.464474,127,74.885522,93.734353,93.721976,6.239403,6.286900,3.097289,3.150706,-0.707066
1,Sin Adiflow,-73.580461,7367.963461,7367.963461,0.000,-0.833437,127,74.932000,95.190133,94.419179,4.795867,5.641938,3.106000,3.124847,-0.998654


In [119]:
cols = ['order', 'Dieta', 'op', 'date', 'Tiene Adiflow', 'panificadas', 'entregadas', 'code_101', 'Producción (Ton)',
       'Anulación (Ton)',
       'diff_prod',  'sackoff_prod', 'peso_agua'
]
df_cut_nov = df_cut_nov.sort_values('sackoff_prod', ascending=True)
df_cut_nov_dep = df_cut_nov[cols]
s3.save_dataframe(df_cut_nov_dep,  "datos_noviembre_sackoff_fazenda.csv")
df_cut_nov_dep

,order,Dieta,op,date,Tiene Adiflow,panificadas,entregadas,code_101,Producción (Ton),Anulación (Ton),diff_prod,sackoff_prod,peso_agua
452,10015982.0,REEMPLAZOS S1B,22075,2025-12-09,Sin Adiflow,10.0,8.590,8.590,10.028288,0.000,-1.43830,-0.143423,0.00000
418,10015899.0,INICIACION P INMUNIDAD,22024,2025-12-02,Con Adiflow,64.0,57.368,57.368,64.286253,0.000,-6.91828,-0.107616,0.30005
431,10015942.0,FINALIZACION,22049,2025-12-04,Con Adiflow,8.0,7.220,7.220,8.032005,0.000,-0.81201,-0.101096,0.07400
289,10015608.0,MACHOS ESP 113,21874,2025-11-01,Sin Adiflow,2.0,1.820,1.820,2.002710,0.000,-0.18271,-0.091231,0.00000
432,10015943.0,ENGORDE ESP P,22052,2025-12-04,Sin Adiflow,48.4,44.170,44.170,48.448962,0.000,-4.27896,-0.088319,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,10015875.0,INICIACION P INMUNIDAD,22014,2025-11-29,Con Adiflow,140.0,79.729,79.729,76.361861,0.000,40.87365,0.044095,0.35375
428,10015939.0,ENGORDE ESP P,22045,2025-12-04,Con Adiflow,64.0,72.778,72.778,68.958491,0.000,3.81951,0.055389,0.63170
459,10015994.0,PREINICIACION F2 P INMUNIDAD,22082,2025-12-11,Sin Adiflow,92.0,97.830,97.830,92.249951,0.000,-52.17145,0.060488,0.00000
365,10015776.0,LACTANCIA PRIMERIZA,21967,2025-11-18,Con Adiflow,110.0,13.800,13.800,12.989000,0.000,-96.87438,0.062437,0.51915


In [120]:

seg_ = ["Tiene Adiflow"]
q = df_cut_nov.groupby(seg_)["sackoff_op"].quantile(
    [0,0.01, 0.02, 0.10, 0.15, 0.20, 0.30, 0.50, 0.90, 0.98, 0.99,1]
).unstack(level=1)

q.columns = ["min","q01", "q02", "q10", 'q15', 'q20', 'q30', "q50", "q90", "q98", "q99", "max"]
q["mean"] = df_cut_nov.groupby(seg_)["sackoff_op"].mean()
#q["min"] = df_cut_nov.groupby(seg_)["sackoff_op"].min()
#q["max"] = df_cut_nov.groupby(seg_)["sackoff_op"].max()
q

,min,q01,q02,q10,q15,q20,q30,q50,q90,q98,q99,max,mean
Tiene Adiflow,,,,,,,,,,,,,
Con Adiflow,-10.761637,-9.030857,-5.777599,-2.216097,-1.850564,-1.598324,-1.093065,-0.464474,0.902030,3.756210,5.233914,6.243745,-0.583208
Sin Adiflow,-14.342308,-9.047422,-8.419175,-4.921394,-3.284247,-2.514825,-1.693010,-0.833437,0.928203,2.837419,3.676094,6.048837,-1.254235


In [45]:
q_min = df_cut_nov["sackoff_op"].quantile(0.10)
q_max = df_cut_nov["sackoff_op"].quantile(0.99)

cond1 = df_cut_nov["sackoff_op"] >= q_min
cond2 = df_cut_nov["sackoff_op"] <= q_max
cond_range = cond1 & cond2

df_cut_nov_new = df_cut_nov[cond_range].copy()
df_bad_nov_new = df_cut_nov[~cond_range].copy()
df_bad_nov_new


,order,Dieta,op,date,panificadas,entregadas,code_101,Anulación (Ton),code_122,code_309,...,peso_agua,Tiene Adiflow,pdi_agro,dureza_agro,finos_agro,product_name,temp1_acond_c,pdi,dureza,finos
452,10015982.0,REEMPLAZOS S1B,22075,2025-12-09,10.0,8.590,8.590,0.0,0.0,0.0,...,0.00000,Sin Adiflow,91.000000,3.000000,9.000000,NaN,NaN,NaN,NaN,NaN
418,10015899.0,INICIACION P INMUNIDAD,22024,2025-12-02,64.0,57.368,57.368,0.0,0.0,0.0,...,0.30005,Con Adiflow,94.400000,3.100000,5.600000,NaN,NaN,NaN,NaN,NaN
431,10015942.0,FINALIZACION,22049,2025-12-04,8.0,7.220,7.220,0.0,0.0,0.0,...,0.07400,Con Adiflow,94.000000,3.300000,6.000000,NaN,NaN,NaN,NaN,NaN
289,10015608.0,MACHOS ESP 113,21874,2025-11-01,2.0,1.820,1.820,0.0,0.0,0.0,...,0.00000,Sin Adiflow,92.900000,3.100000,7.100000,MACHOS P,90.0,92.90,3.1,7.10
432,10015943.0,ENGORDE ESP P,22052,2025-12-04,48.4,44.170,44.170,0.0,0.0,0.0,...,0.00000,Sin Adiflow,94.200000,3.200000,5.800000,NaN,NaN,NaN,NaN,NaN
422,10015907.0,REEMPLAZOS S1B,22040,2025-12-02,12.0,11.053,11.053,0.0,0.0,0.0,...,0.00000,Sin Adiflow,95.400000,3.400000,4.600000,NaN,NaN,NaN,NaN,NaN
481,10016044.0,GESTACION ESP P,22104,2025-12-16,32.0,29.770,29.770,0.0,0.0,0.0,...,0.00000,Sin Adiflow,91.400000,3.100000,8.600000,NaN,NaN,NaN,NaN,NaN
449,10015977.0,FLUSHING,22069,2025-12-09,26.0,26.260,26.260,0.0,0.0,0.0,...,0.12950,Con Adiflow,96.600000,3.100000,3.400000,NaN,NaN,NaN,NaN,NaN
491,10016079.0,INICIACION P INMUNIDAD,22120,2025-12-18,56.0,52.766,52.766,0.0,0.0,0.0,...,0.00000,Sin Adiflow,95.100000,3.150000,4.900000,NaN,NaN,NaN,NaN,NaN
474,10016023.0,NOVILLONAS SUPLEMENTO,22097,2025-12-15,6.0,5.660,5.660,0.0,0.0,0.0,...,0.00000,Sin Adiflow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
df_cut_nov_dep[df_cut_nov_dep["op"]=="22091"]

,order,Dieta,op,date,Tiene Adiflow,panificadas,entregadas,code_101,Producción (Ton),Anulación (Ton),diff_prod,sackoff_prod,peso_agua


In [130]:
df_bad[df_bad["op"]=="22091"]

,order,Dieta,op,date,panificadas,entregadas,code_101,Anulación (Ton),code_122,code_309,...,peso_agua,Tiene Adiflow,pdi_agro,dureza_agro,finos_agro,product_name,temp1_acond_c,pdi,dureza,finos
468,10016011.0,MACHOS ESP 113,22091,2025-12-23,2.3,2.3,2.3,0.0,0.0,0.0,...,0.0,Sin Adiflow,90.8,3.0,9.2,NaN,NaN,NaN,NaN,NaN
